# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-03-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-01-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-17-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-13-events.csv', '/home/workspace/event_data/2018-11-27-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# Initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
#             print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster

try: 
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS my_keyspace
    WITH REPLICATION = 
    {
        'class': 'SimpleStrategy',
        'replication_factor': 1
    }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('my_keyspace')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# Reading the newly created CSV file into a dataframe
df = pd.read_csv('event_datafile_new.csv')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,A Fine Frenzy,Anabelle,F,0,Simpson,267.91138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",256,Almost Lover (Album Version),69
1,Nirvana,Aleena,F,0,Kirby,214.77832,paid,"Waterloo-Cedar Falls, IA",237,Serve The Servants,44
2,Television,Aleena,F,1,Kirby,238.49751,paid,"Waterloo-Cedar Falls, IA",237,See No Evil (Remastered LP Version),44
3,JOHN COLTRANE,Aleena,F,2,Kirby,346.43546,paid,"Waterloo-Cedar Falls, IA",237,Blues To Bechet (LP Version),44
4,NOFX,Aleena,F,3,Kirby,80.79628,paid,"Waterloo-Cedar Falls, IA",237,It's My Job To Keep Punk Rock Elite,44


In [9]:
# Get the shape of the dataframe
df.shape

(6820, 11)

In [10]:
# Get the describe of the dataframe
df.describe()

,itemInSession,length,sessionId,userId
count,6820.000000,6820.000000,6820.000000,6820.000000
mean,22.761144,247.032221,599.181818,54.681232
std,23.444636,102.975921,284.953333,28.162734
min,0.000000,15.855870,3.000000,2.000000
25%,4.000000,197.321998,374.000000,29.000000
50%,15.000000,232.972605,605.000000,49.000000
75%,35.000000,274.121992,834.000000,80.000000
max,126.000000,2594.873020,1114.000000,101.000000


## Query 1:  
#### Get the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [11]:
# Performing a query on the dataframe - This will also act as the expected result from the database query
df[(df.sessionId == 338) & (df.itemInSession == 4)]  

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
6376,Faithless,Ava,F,4,Robinson,495.3073,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50


In [12]:
# Creating the table music_library_1
try:
    session.execute("""CREATE TABLE IF NOT EXISTS music_library_1 
                            (sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))""")
except Exception as e:
    print(e)

In [13]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library_1 (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
#         print(line)
        
        # To make this easier to read I have assigned variables to the result of line, instead of using line[0], line[1], etc.
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        
        session.execute(query, (int(sessionId), int(itemInSession), str(artist), str(song), float(length)))

#### Do a SELECT to verify that the data have been inserted into the table

In [14]:
# SELECT statement to verify the data was entered into the table
try:
    rows = session.execute("""
    SELECT * 
    FROM music_library_1 
    WHERE sessionId = 338 
    AND itemInSession = 4
    """)
except Exception as e:
    print(e)
    
for row in rows:
#     print(row)
    print(f" Artist: {row.artist}, Song title: {row.song}, Song length: {row.length}")

 Artist: Faithless, Song title: Music Matters (Mark Knight Dub), Song length: 495.30731201171875


## Query 2: 
#### Get the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [15]:
# Performing a query on the dataframe - This will also act as the expected result from the database query
df[(df.userId == 10) & (df.sessionId == 182)]          

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
5141,Down To The Bone,Sylvie,F,0,Cruz,333.76608,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Keep On Keepin' On,10
5142,Three Drives,Sylvie,F,1,Cruz,411.63710,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Greece 2000,10
5143,Sebastien Tellier,Sylvie,F,2,Cruz,377.73016,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Kilometer,10
5144,Lonnie Gordon,Sylvie,F,3,Cruz,181.21098,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Catch You Baby (Steve Pitron & Max Sanna Radio...,10


In [16]:
# Creating a copy of the current dataframe
new_df = df.copy()

In [17]:
# Creating a user column on a dataframe
new_df['user'] = new_df.firstName + " " + new_df.lastName

In [18]:
# Export new dataframe to csv
new_df.to_csv('event_data_query_2.csv', index=False)

In [19]:
# Performing a query on the new dataframe - This will also act as the expected result from the database query
new_df[(new_df.userId == 10) & (new_df.sessionId == 182)] 

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId,user
5141,Down To The Bone,Sylvie,F,0,Cruz,333.76608,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Keep On Keepin' On,10,Sylvie Cruz
5142,Three Drives,Sylvie,F,1,Cruz,411.63710,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Greece 2000,10,Sylvie Cruz
5143,Sebastien Tellier,Sylvie,F,2,Cruz,377.73016,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Kilometer,10,Sylvie Cruz
5144,Lonnie Gordon,Sylvie,F,3,Cruz,181.21098,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Catch You Baby (Steve Pitron & Max Sanna Radio...,10,Sylvie Cruz


In [20]:
# Creating the table music_library_2
try:
    session.execute("""CREATE TABLE IF NOT EXISTS music_library_2 
        (userId int, sessionId int, itemInSession int, user text, artist text, song text, 
            PRIMARY KEY ((userId), sessionId, itemInSession, user))""")
except Exception as e:
    print(e)

In [21]:
# Using the newly created csv file with user column
file = 'event_data_query_2.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library_2 (userId, sessionId, itemInSession, user, artist, song)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
#         print(line)
        
        # To make this easier to read I have assigned variables to the result of line, instead of using line[0], line[1], etc.
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId, user = line
        
        session.execute(query, (int(userId), int(sessionId), int(itemInSession), str(user), str(artist), str(song)))

#### Do a SELECT to verify that the data have been inserted into the table

In [22]:
# SELECT statement to verify the data was entered into the table
try:
    rows = session.execute("""
    SELECT * 
    FROM music_library_2 
    WHERE userId = 10 
    AND sessionId = 182
    """)
except Exception as e:
    print(e)
    
for row in rows:
#     print(row)
    print(f" Artist: {row.artist}, Song title: {row.song}, Item in Session: {row.iteminsession}, User: {row.user}")

 Artist: Down To The Bone, Song title: Keep On Keepin' On, Item in Session: 0, User: Sylvie Cruz
 Artist: Three Drives, Song title: Greece 2000, Item in Session: 1, User: Sylvie Cruz
 Artist: Sebastien Tellier, Song title: Kilometer, Item in Session: 2, User: Sylvie Cruz
 Artist: Lonnie Gordon, Song title: Catch You Baby (Steve Pitron & Max Sanna Radio Edit), Item in Session: 3, User: Sylvie Cruz


## Query 3: 
#### Get every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [23]:
# Performing a query on the dataframe - This will also act as the expected result from the database query
new_df[new_df.song == 'All Hands Against His Own']        

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId,user
1167,The Black Keys,Sara,F,31,Johnson,196.91057,paid,"Winston-Salem, NC",152,All Hands Against His Own,95,Sara Johnson
2148,The Black Keys,Tegan,F,25,Levine,196.91057,paid,"Portland-South Portland, ME",611,All Hands Against His Own,80,Tegan Levine
3132,The Black Keys,Jacqueline,F,50,Lynch,196.91057,paid,"Atlanta-Sandy Springs-Roswell, GA",559,All Hands Against His Own,29,Jacqueline Lynch


In [24]:
# Creating the table music_library_3
try:
    session.execute("""CREATE TABLE IF NOT EXISTS music_library_3 
        (song text, userId int, user text, firstName text, lastName text, PRIMARY KEY ((song), userId, user))""")
except Exception as e:
    print(e)

In [25]:
# Using the newly created csv file with user column
file = 'event_data_query_2.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library_3 (song, userId, user, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
#         print(line)
        
        # To make this easier to read I have assigned variables to the result of line, instead of using line[0], line[1], etc.
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId, user = line
        
        session.execute(query, (str(song), int(userId), str(user), str(firstName), str(lastName)))

#### Do a SELECT to verify that the data have been inserted into the table

In [26]:
# SELECT statement to verify the data was entered into the table
try:
    rows = session.execute("""
    SELECT * 
    FROM music_library_3 
    WHERE song = 'All Hands Against His Own'
    """)
except Exception as e:
    print(e)
    
for row in rows:
#     print(row)
    print(f" User: {row.user}, First Name: {row.firstname}, Surname: {row.lastname}")

 User: Jacqueline Lynch, First Name: Jacqueline, Surname: Lynch
 User: Tegan Levine, First Name: Tegan, Surname: Levine
 User: Sara Johnson, First Name: Sara, Surname: Johnson


### Drop the tables before closing out the sessions

In [27]:
session.execute("DROP TABLE music_library_1")
session.execute("DROP TABLE music_library_2")
session.execute("DROP TABLE music_library_3")

### Close the session and cluster connection

In [28]:
session.shutdown()
cluster.shutdown()